#PCA & CNN QUIZ 1
2301931251 - Daffa Rizki Rizaly

In [24]:
from keras.datasets import cifar10
import matplotlib.pyplot as pyplot
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from keras.datasets import cifar10

In [2]:
(trainX, trainy), (testX, testy) = cifar10.load_data()

In [3]:
trainX[5][12][22]

array([201,  72,  69], dtype=uint8)

In [4]:
trainX.shape

(50000, 32, 32, 3)

In [5]:
x_train = trainX.reshape(trainX.shape[0],-1)
x_test = testX.reshape(testX.shape[0], -1)
y_train = trainy
y_test = testy

In [6]:
pca = PCA()
pca.fit_transform(x_train)

array([[-1.63225950e+03,  6.95904885e+02,  3.82936183e+02, ...,
        -1.94383396e-01, -2.96716275e-01, -8.91909421e-02],
       [ 2.11594763e+02, -2.42235521e+02,  1.53095691e+03, ...,
         2.76508201e-02,  1.47062424e-01, -1.96606511e-01],
       [ 1.97120094e+03, -2.93813609e+03, -7.02173229e+02, ...,
        -2.66373278e-01, -5.45969636e-02, -1.00334729e-01],
       ...,
       [ 1.80373380e+02, -2.89234292e+03, -1.00448294e+02, ...,
        -9.69870999e-02,  4.75402487e-01,  3.69202335e-01],
       [ 2.89881680e+03, -8.60874301e+02,  1.41938040e+03, ...,
        -1.19109692e-01,  1.89968105e-01, -4.26015893e-01],
       [ 1.06916310e+03, -3.01968063e+02, -1.13372056e+03, ...,
         2.46831906e-01,  6.12928127e-02, -7.15290870e-03]])

In [7]:
# Calculating optimal k to have 95% (say) variance 

k = 0
total = sum(pca.explained_variance_)
current_sum = 0

while(current_sum / total < 0.99):
    current_sum += pca.explained_variance_[k]
    k += 1
k

658

In [8]:
## Applying PCA with k calcuated above

pca = PCA(n_components=k, whiten=True)

x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [9]:
## Creating a model

from keras.models import Sequential
model = Sequential()

In [10]:
from  keras.layers  import Dense

# For first layer, input shape must be supplied
layer1 = Dense(units = 50, activation = 'relu', input_dim = k)
model.add(layer1)

layer2 = Dense(units = 25, activation = 'relu')
model.add(layer2)

layer3 = Dense(units = 10, activation = 'sigmoid')
model.add(layer3)

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])


In [12]:
## Onehot Encoding the labels
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
encoder = OneHotEncoder(sparse=False)
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [13]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [16]:
model.fit(x_train_pca, y_train, epochs=50)

Epoch 1/50
1563/1563 [==============================] - 17s 2ms/step - loss: 0.3575 - accuracy: 0.1960
Epoch 2/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2477 - accuracy: 0.4402
Epoch 3/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2289 - accuracy: 0.4872
Epoch 4/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2132 - accuracy: 0.5319
Epoch 5/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2015 - accuracy: 0.5622
Epoch 6/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.1937 - accuracy: 0.5806
Epoch 7/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.1865 - accuracy: 0.6011
Epoch 8/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.1794 - accuracy: 0.6209
Epoch 9/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.1750 - accuracy: 0.6324
Epoch 10/50
1563/1563 [==============================] - 3s 2ms/step - loss: 0.17

In [17]:
predictions = model.predict(x_test_pca)


In [18]:
predictions_decoded = []

for x in predictions:
    for i in range(x.shape[0]):
        if x[i] == max(x):
            predictions_decoded.append(i)
            break